In [36]:
from bs4 import BeautifulSoup
import ujson as json
import requests

In [57]:
list_url = "http://terms.naver.com/list.nhn?cid=51007&page=%d&categoryId=51007"
mobile_root = "http://m.terms.naver.com"
entries = []

page = 1
while True:
    print(page)
    
    res = requests.get(list_url%page)
    b = BeautifulSoup(res.text, 'lxml')
    
    current_page = int(b.find('div', {'class': 'paginate'}).find('strong').text)
    if page > current_page:
        # stop if there is no more pages to crawl
        break
    
    for entry in b.find('ul', {'class': 'content_list'}).findAll('li'):
        url = entry.findAll('a')[0]['href']
        res = requests.get(mobile_root+url)
        b = BeautifulSoup(res.text, 'lxml')
        
        entry = dict()
        title = None

        div_content = b.find('div', {'class': '_SLOG_TERMS_CONTENT'})
        additional_info = div_content.find('ul', {'class': 'na_property_table2'})
        if additional_info is not None:
            for li in additional_info.findAll('li'):
                entry[li.find('em').text] = li.find('span').text

        div_content = div_content.findAll('div', {'class': 'atomic-block'})
        title_group = div_content[0].find('div', {'class': 'na_title_group'})
        entry['ko_name'] = title_group.find('h1').text
        try:
            entry['name'] = title_group.find('p').text.replace('\xa0', ' ')
        except:
            pass
        
        abstract = div_content[0].find('div', {'class': 'na_abstract'})
        if abstract is not None:
            abstract.find('strong').extract()
            entry['abstract'] = abstract.text.strip()
            
        for div in div_content[1:]:
            if "breakable-block" in div['class']:
                entry[title] = div.text.strip()
            else:
                title = div.findChild()
                if 'na_section_title' in title['class']:
                    title = div.text
                else:
                    # picture div
                    continue
                
        entries.append(entry)

    page += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91


In [58]:
with open('result.json', 'w') as f:
    json.dump(entries, f)

In [59]:
entries[0]

{'abstract': '사람 몸의 등뼈 부근에 있는 신경절에 잠복상태로 있던 수두 바이러스가 다시 활성화되면서 발생하는 질병',
 'ko_name': '대상포진',
 'name': 'herpes zoster',
 '검사': '수포를 면봉으로 긁어서 대상포진 바이러스에 감염된 특징적인 인체세포 모양이 관찰되면 대상포진을 의심할 수 있고, 수포액을 세포 배양하여 바이러스를 검출하여 이를 확인할 수 있다. 바이러스의 핵산을 검출하는 중합효소연쇄반응(PCR; polymerase chain reaction)도 임상에서 사용되고 있다. 이러한 검사들은 일반적인 환자들에서는 검사 비용에 비해 그 유용성이 적기 때문에 잘 시행하지 않지만, 입원환자의 경우에는 검사가 필요할 수 있다.',
 '경과/합병증': '면역억제환자에서는 대상포진이 지각신경이 분포하는 부위에 국한되지 않고 전신의 피부에 나타나기도 하며, 뇌수막염이나 뇌염으로 진행하거나 간염이나 폐렴을 일으켜 사망에 이르는 경우도 있다. 증상이 좋아지더라도 바이러스는 잠복상태로 몸 속에 계속 존재하기 때문에 바이러스가 재활성화 되면 다시 대상포진이 발생할 수 있다.',
 '관련 신체기관': '피부, 신경계',
 '관련 증상': '농포, 소수포, 작열감, 홍반, 수포',
 '원인': '그러나 신체의 면역력이 약해지면 신경절에 잠복해 있던 수두 바이러스가 신경을 타고 다시 피부로 내려와 그 곳에서 염증을 일으키거나, 심하면 염증이 전신으로 퍼질 수도 있다. 이러한 증상의 원인이 되는 바이러스는 수두 바이러스와 동일하지만 이 경우에는 대상포진 바이러스라고 부른다. 수두-대상포진 바이러스라고 묶어서 부르기도 한다.',
 '정의': '* 발병위치 : 피부, 신경계',
 '증상': '수포는 10∼14일 동안 변화하는데, 고름이 차면서 탁해지다가 딱지로 변하게 된다. 접촉 등에 의해 물집이 터지면 궤양이 형성될 수 있다. 보통 2주 정도 지나면 딱지가 생기면서 증상이 좋아진다. 피부의 병적인 증상이 모두 좋아진 후에도 해당 부위가 

In [60]:
len(entries)

1344